In [2]:
%pip install Levenshtein numpy pandas matplotlib scipy tqdm plotly

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
import functools
import itertools
import random

from collections import namedtuple
import heapq
from typing import List, Tuple, Any

import numpy as np
import pandas as pd
import Levenshtein

import matplotlib.pyplot as plt
import plotly.figure_factory as figure_factory

from tqdm import tqdm
from time import perf_counter


In [95]:
NamedHeuristic = namedtuple("NamedHeuristic", ["name", "callable"])

class Node:
    """
    A class to represent a node in the puzzle.

    Attributes:
    -----------
    board: numpy.ndarray
        An array of integers representing the current state of the puzzle board.
    parent: Node
        A reference to the parent node.
    cost: int
        The cost of the current node.
    """

    def __init__(self, board: np.ndarray, parent: Any) -> None:
        """
        Constructs all the necessary attributes for the Node object.

        Parameters:
        -----------
        board: numpy.ndarray
            An array of integers representing the current state of the puzzle board.
        parent: Any
            A reference to the parent node.
        """
        self.board = board
        self.parent = parent
        self.cost = 0

    @property
    def empty_pos(self) -> Tuple[int, int]:
        """
        The position of the empty slot in the puzzle.

        Returns:
        --------
        Tuple[int, int]:
            The position of the empty slot in the puzzle.
        """
        y, x = np.where(self.board == 0)
        x = x.item()
        y = y.item()
        return (y, x)

    @property
    def signature(self) -> str:
        """
        The signature of the current state of the puzzle.

        Returns:
        --------
        str:
            The signature of the current state of the puzzle.
        """
        flatten_board = list(itertools.chain.from_iterable(self.board))
        list_to_signature = lambda a, b: str(a) + "-" + str(b)
        return functools.reduce(list_to_signature, flatten_board)

    @property
    def children(self) -> List['Node']:
        """
        The children of the current node.

        Returns:
        --------
        List[Node]:
            The children of the current node.
        """
        children = []
        possible_moves = self.__get_possible_moves()
        for move in possible_moves:
            child_board = self.apply_move(move)
            child = Node(child_board, self)
            children.append(child)
        return children

    def __get_possible_moves(self) -> List[Tuple[int, int]]:
        """
        The possible moves from the current state of the puzzle.

        Returns:
        --------
        List[Tuple[int, int]]:
            The possible moves from the current state of the puzzle.
        """
        all_moves = [(0, 1), (1, 0), (0, -1), (-1, 0)]
        x, y = self.empty_pos
        possible_moves = []
        for x_delta, y_delta in all_moves:
            new_x = x + x_delta
            new_y = y + y_delta
            if (-1 < new_x < 4) and ((-1 < new_y < 4)):
                move = (new_x, new_y)
                possible_moves.append(move)
        return possible_moves

    def apply_move(self, move: Tuple[int, int]) -> np.ndarray:
        """
        Applies the given move to the current state of the puzzle.

        Parameters:
        -----------
        move: Tuple[int, int]
            The move to apply.

        Returns:
        --------
        numpy.ndarray:
            The new state of the puzzle after applying the move.
        """
        new_board = self.board.copy()
        new_board[self.empty_pos], new_board[move] = (
            new_board[move],
            new_board[self.empty_pos],
        )
        return new_board


def generate_random_game() -> Node:
    """
    Generates a random game by performing a random sequence of
    moves starting from the solved game state.

    :return: the final node of the random game
    """
    tiles = list(range(1, 16))
    tiles.append(0)
    board = np.array(np.array_split(tiles, 4))
    node = Node(board=board, parent=None)

    for _ in range(random.randint(5000, 10000)):
        node = random.choice(node.children)
    node.parent = None
    return node


def print_solution(initial_node: Node, end_node: Node) -> None:
    """
    Prints the solution path from the initial node to the end node.

    :param initial_node: the initial node of the game
    :param end_node: the end node of the game
    """
    solution = []
    while end_node.parent:
        solution.append(end_node.board)
        end_node = end_node.parent
    solution = solution[::-1]
    print(f"{initial_node.board}\n")
    for board in solution:
        print(f"{board}\n")


def get_node_distance_from_parent(end_node: Node) -> int:
    """
    Returns the number of steps from the end node to its parent.

    Args:
    - end_node: The node to start the distance calculation from.

    Returns:
    - The number of steps from the end node to its parent.
    """
    i = 0
    while end_node.parent:
        end_node = end_node.parent
        i += 1
    return i


def levenshtein_distance(node: Node) -> int:
    """
    Calculates the Levenshtein distance between the node's signature and the solution signature.

    Args:
        node (Node): The node to calculate the Levenshtein distance for.

    Returns:
        int: The Levenshtein distance.
    """
    solution_signature =  "1-2-3-4-5-6-7-8-9-10-11-12-13-14-15-0"
    return Levenshtein.distance(node.signature, solution_signature)


def misplaced_tiles(node: Node) -> int:
    """
    Calculates the number of misplaced tiles between the node's signature and the solution signature.
    The cost of the misplaced tiles is weighted by the tile's position.

    Args:
        node (Node): The node to calculate the misplaced tiles for.

    Returns:
        int: The cost of the misplaced tiles.
    """
    solution_signature =  "1-2-3-4-5-6-7-8-9-10-11-12-13-14-15-0"

    first_tiles_penalty = 9
    cost = 0
    for i, j in zip(solution_signature.split("-"), node.signature.split("-")):
        if i != j:
            cost += 1 + first_tiles_penalty
            first_tiles_penalty -= 1
    return cost


def gradient(node: Node) -> int:
    """
    Calculates the gradient between the node's signature and the solution signature.

    Args:
        node (Node): The node to calculate the gradient for.

    Returns:
        int: The gradient cost.
    """
    solution_signature =  "1-2-3-4-5-6-7-8-9-10-11-12-13-14-15-0"
    cost = 0
    for i, j in zip(solution_signature.split("-"), node.signature.split("-")):
        cost += abs(int(i) - int(j))
    return cost


def a_star(initial_node: Node, heuristic: callable) -> Node:
    """
    Find the shortest path from an initial node to a node with signature "123456780" using A* algorithm.

    Args:
        initial_node: The starting node of the puzzle.
        heuristic: A callable that takes a node as input and returns an estimate of the remaining
                   distance to the target node.

    Returns:
        The target node with signature "123456780".
    """
    target_signature =  "1-2-3-4-5-6-7-8-9-10-11-12-13-14-15-0"
    visited_nodes = set()

    if initial_node.signature == target_signature:
        return initial_node

    queue = [(heuristic(initial_node), initial_node)]
    heapq.heapify(queue)
    visited_nodes.add(initial_node.signature)

    while len(queue) > 0:
        _, node = heapq.heappop(queue)

        if node.signature == target_signature:
            return node

        for child_node in node.children:
            if child_node.signature in visited_nodes:
                continue
            child_node.cost = heuristic(child_node) + len(visited_nodes) ** 2.5 + node.cost
            visited_nodes.add(child_node.signature)
            heapq.heappush(queue, (child_node.cost, child_node))

In [97]:
heuristics = [
    NamedHeuristic("Levenshtein", levenshtein_distance),
    NamedHeuristic("Misplaced Tiles", misplaced_tiles),
    NamedHeuristic("Gradient", gradient)
]

games = []
pb = tqdm([generate_random_game() for _ in range(100)], leave=True, position=0)
for initial_node in pb:
    for heuristic in heuristics:
        pb.set_description(f"Running for {initial_node.signature} {heuristic.name}")
        t0 = perf_counter()
        end_node = a_star(initial_node, heuristic.callable)
        delta = perf_counter() - t0 
        
        games.append(
            {
            "initial_node": initial_node.signature,
            "distance": get_node_distance_from_parent(end_node),
            "time_spent": delta,
            "heuristic": heuristic.name
            }
        )

games = pd.DataFrame(games)

Running for 13-1-9-12-7-6-5-4-14-11-15-0-3-10-8-2 Levenshtein:   0%|          | 0/100 [00:00<?, ?it/s]

In [ ]:
levenshtein = games.query("heuristic == 'Levenshtein'")["time_spent"].values
misplaced = games.query("heuristic == 'Misplaced Tiles'")["time_spent"].values
gradient = games.query("heuristic == 'Gradient'")["time_spent"].values

hist_data = [levenshtein, misplaced, gradient]
group_labels = ["Levenshtein", "Misplaced Tiles", "Gradient"]

fig = figure_factory.create_distplot(hist_data, group_labels)
fig.update_layout(title_text='Distribution of time spent finding the solution', xaxis_title="time spent calculating")
fig.show()

In [ ]:
levenshtein = games.query("heuristic == 'Levenshtein'")["distance"].values
misplaced = games.query("heuristic == 'Misplaced Tiles'")["distance"].values
gradient = games.query("heuristic == 'Gradient'")["distance"].values

hist_data = [levenshtein, misplaced, gradient]
group_labels = ["Levenshtein", "Misplaced Tiles", "Gradient"]

fig = figure_factory.create_distplot(hist_data, group_labels)
fig.update_layout(title_text='Distribution of moves took finding the solution', xaxis_title="number of moves")
fig.show()

In [91]:
games.time_spent.mean()

6.584779833316679